In [19]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import re
import pandas as pd
import numpy as np
import math

In [14]:
# url com o bairro agua verde
url = 'https://www.vivareal.com.br/aluguel/parana/curitiba/bairros/agua-verde/apartamento_residencial/#onde=BR-Parana-NULL-Curitiba-Barrios-Agua_Verde&tipos=apartamento_residencial'
page_content = requests.get(url).content

In [16]:
f = open('vivarealtest.html', 'x')

In [17]:
f.write(str(page_content))
f.close()

In [18]:
soup = BeautifulSoup(page_content, 'html.parser')

In [21]:
cards = soup.find_all('div', 'property-card__main-content')

In [24]:
len(cards)

36

In [82]:
cards_info = []
for card in cards:
    link_detalhes = card.find('a', 'property-card__title js-cardLink js-card-title')['href']
    id = re.search('id-(\d+)', link_detalhes).group(1)
    titulo = card.find('a', 'property-card__title js-cardLink js-card-title').text.strip()
    tamanho = card.find('span', 'property-card__detail-value js-property-card-value property-card__detail-area js-property-card-detail-area').text.strip()
    quartos = card.find('li', 'property-card__detail-item property-card__detail-room js-property-detail-rooms').find('span', 'property-card__detail-value js-property-card-value').text.strip()
    suites = card.find('li', 'property-card__detail-item property-card__detail-item-extra js-property-detail-suites').find('span', 'property-card__detail-value js-property-card-value').text.strip()
    banheiros = card.find('li', 'property-card__detail-item property-card__detail-bathroom js-property-detail-bathroom').find('span', 'property-card__detail-value js-property-card-value').text.strip()
    vagas_carro = card.find('li', 'property-card__detail-item property-card__detail-garage js-property-detail-garages').find('span', 'property-card__detail-value js-property-card-value').text.strip()
    valor_aluguel = card.find('div', 'property-card__price js-property-card-prices js-property-card__price-small').text.strip().split()[1].replace('.', '')
    valor_condominio = card.find('div', 'property-card__price-details--condo')
    if valor_condominio:
        valor_condominio = valor_condominio.find('strong', 'js-condo-price').text.strip().split()[1].replace('.', '')
    else:
        valor_condominio = None
    cards_info.append({'id': id, 'titulo': titulo, 'link_detalhes': link_detalhes, 'tamanho': tamanho, 'quartos': quartos,
                      'suites': suites, 'banheiros': banheiros, 'vagas_carro': vagas_carro, 
                       'valor_aluguel': valor_aluguel, 'valor_condominio': valor_condominio})

In [83]:
cards_info

[{'id': '2490462708',
  'titulo': 'Apartamento com 3 Quartos para Aluguel, 173m²',
  'link_detalhes': '/imovel/apartamento-3-quartos-agua-verde-bairros-curitiba-com-garagem-173m2-aluguel-RS3000-id-2490462708/',
  'tamanho': '173',
  'quartos': '3',
  'suites': '1',
  'banheiros': '3',
  'vagas_carro': '2',
  'valor_aluguel': '3000',
  'valor_condominio': '700'},
 {'id': '2491073478',
  'titulo': 'Apartamento com 2 Quartos para Aluguel, 99m²',
  'link_detalhes': '/imovel/apartamento-2-quartos-agua-verde-bairros-curitiba-com-garagem-99m2-aluguel-RS2300-id-2491073478/',
  'tamanho': '99',
  'quartos': '2',
  'suites': '1',
  'banheiros': '3',
  'vagas_carro': '1',
  'valor_aluguel': '2300',
  'valor_condominio': '650'},
 {'id': '2493130017',
  'titulo': 'Apartamento com 4 Quartos para Aluguel, 350m²',
  'link_detalhes': '/imovel/apartamento-4-quartos-agua-verde-bairros-curitiba-com-garagem-350m2-aluguel-RS6700-id-2493130017/',
  'tamanho': '350',
  'quartos': '4',
  'suites': '4',
  'banh

In [94]:
for card_info in cards_info:
    url = 'https://www.vivareal.com.br/' + card_info['link_detalhes']
    page_content = requests.get(url).content
    soup = BeautifulSoup(page_content, 'html.parser')
    descricao = soup.find('p', 'description__text').text.strip()
    card_info['descricao'] = descricao

In [95]:
cards_info

[{'id': '2490462708',
  'titulo': 'Apartamento com 3 Quartos para Aluguel, 173m²',
  'link_detalhes': '/imovel/apartamento-3-quartos-agua-verde-bairros-curitiba-com-garagem-173m2-aluguel-RS3000-id-2490462708/',
  'tamanho': '173',
  'quartos': '3',
  'suites': '1',
  'banheiros': '3',
  'vagas_carro': '2',
  'valor_aluguel': '3000',
  'valor_condominio': '700',
  'descricao': 'Bom apto no Batel, ensolarado, totalmente mobiliado, com 173 m² de área privativa, 3 quartos, 2 vagas grandes paralelas, 1 suíte master com closet e hidro. Banheiro social, lavabo, sala de jantar e estar, sala de TV com lareira e sacada. Armários embutidos no closet da suíte master e no corredor. 4º andar. Cozinha com armários, área de serviço com dependência de empregada. Edifício com portaria e salão de festas. Excelente região, próximo a restaurantes, mercados e bancos.'},
 {'id': '2491073478',
  'titulo': 'Apartamento com 2 Quartos para Aluguel, 99m²',
  'link_detalhes': '/imovel/apartamento-2-quartos-agua-ve

Bairros:
- agua verde
- alto da XV
- batel
- bigorrilho
- centro
- champagnat
- cristo rei
- jardim botanico
- merces
- rebouças
- sao francisco
- vila izabel

In [2]:
# 01/09 - 3320 resultados
url = 'https://www.vivareal.com.br/aluguel/parana/curitiba/bairros/agua-verde/#onde=BR-Parana-NULL-Curitiba-Barrios-Agua_Verde,BR-Parana-NULL-Curitiba-Barrios-Alto_Da_Rua_Xv,BR-Parana-NULL-Curitiba-Barrios-Alto_da_Rua_XV,BR-Parana-NULL-Curitiba-Barrios-Batel,BR-Parana-NULL-Curitiba-Barrios-Bigorrilho,BR-Parana-NULL-Curitiba-Barrios-Centro,BR-Parana-NULL-Curitiba-Barrios-Cristo_Rei,BR-Parana-NULL-Curitiba-Barrios-Jardim_Botanico,BR-Parana-NULL-Curitiba-Barrios-Merces,BR-Parana-NULL-Curitiba-Barrios-Champagnat,BR-Parana-NULL-Curitiba-Barrios-Reboucas,BR-Parana-NULL-Curitiba-Barrios-Sao_Francisco,BR-Parana-NULL-Curitiba-Barrios-Vila_Izabel&tipos=apartamento_residencial'

In [8]:
pagina = 1
response = requests.get(url)
while response.status_code == 200:
    with open('paginas_html/{}_pag{}.html'.format('viva_real', pagina), 'w') as html_file:
        html_file.write(response.text)
    pagina += 1
    url = 'https://www.vivareal.com.br/aluguel/parana/curitiba/bairros/agua-verde/?pagina={}#onde=BR-Parana-NULL-Curitiba-Barrios-Agua_Verde,BR-Parana-NULL-Curitiba-Barrios-Alto_Da_Rua_Xv,BR-Parana-NULL-Curitiba-Barrios-Alto_da_Rua_XV,BR-Parana-NULL-Curitiba-Barrios-Batel,BR-Parana-NULL-Curitiba-Barrios-Bigorrilho,BR-Parana-NULL-Curitiba-Barrios-Centro,BR-Parana-NULL-Curitiba-Barrios-Cristo_Rei,BR-Parana-NULL-Curitiba-Barrios-Jardim_Botanico,BR-Parana-NULL-Curitiba-Barrios-Merces,BR-Parana-NULL-Curitiba-Barrios-Champagnat,BR-Parana-NULL-Curitiba-Barrios-Reboucas,BR-Parana-NULL-Curitiba-Barrios-Sao_Francisco,BR-Parana-NULL-Curitiba-Barrios-Vila_Izabel&tipos=apartamento_residencial'.format(pagina)
    response = requests.get(url)

In [6]:
response = requests.get(url)

In [7]:
response.text

' <!DOCTYPE html> <html lang="pt-BR"> <head> <script> var pageCategory = \'RESULT PAGE\';\n\n  const clickstreamTracker = function (rawData) {\n    const clickstreamTrackerEndpoint = \'https://tracking.vivareal.com/events/v2\';\n    const data = JSON.stringify(rawData);\n\n    if (navigator.sendBeacon) {\n      navigator.sendBeacon(clickstreamTrackerEndpoint, data);\n    } else {\n      const ajax = new XMLHttpRequest();\n      ajax.open(\'POST\', clickstreamTrackerEndpoint, true);\n      ajax.send(data);\n    }\n  };\n\n  (function () {\n    const getCookie = function(name) {\n      const value = `; ${document.cookie}`;\n      const parts = value.split(`; ${name}=`);\n\n      if (parts.length == 2) return parts.pop().split(\';\').shift();\n      return \'\';\n    }\n\n    const getExperimentId = function() {\n      const gaExp = getCookie(\'_gaexp\');\n\n      if (gaExp) {\n        return gaExp.split(\'!\')\n          .map(v => v.replace(/GAX([0-9]+\\.){2}/, \'\').split(\'.\')[0])\n  

# To do:
Checar por quê estao sendo salvos tantos arquivos html

Dentro das páginas específicas, possuem ammenieties, podem virar features iniciais também

In [27]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome("chromedriver", options=options)

url = "https://www.vivareal.com.br/aluguel/parana/curitiba/bairros/agua-verde/apartamento_residencial/?__vt=srl%3Ab#onde=BR-Parana-NULL-Curitiba-Barrios-Agua_Verde&tipos=apartamento_residencial,flat_residencial,cobertura_residencial"

driver.implicitly_wait(30)
driver.get(url)

soup = BeautifulSoup(driver.page_source, 'html.parser')
total_results = soup.find('strong', {'class' : 'results-summary__count js-total-records'}).text

cards_info = []
# 36 resultados por pagina, faz o teto para garantir que acessou todas as paginas
for i in range(2, math.ceil(int(total_results)/36)+2):
    print(i)
    cards = soup.find_all('div', 'property-card__main-content')
    
    for card in cards:
        link_detalhes = card.find('a', 'property-card__title js-cardLink js-card-title')['href']
        id = re.search('id-(\d+)', link_detalhes).group(1)
        cards_info.append(id)
    
    url = "https://www.vivareal.com.br/aluguel/parana/curitiba/bairros/agua-verde/apartamento_residencial/?__vt=srl%3Ab&pagina={}#onde=BR-Parana-NULL-Curitiba-Barrios-Agua_Verde&tipos=apartamento_residencial,flat_residencial,cobertura_residencial".format(i)
    print(url)
    driver.implicitly_wait(30)
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

2
https://www.vivareal.com.br/aluguel/parana/curitiba/bairros/agua-verde/apartamento_residencial/?__vt=srl%3Ab&pagina=2#onde=BR-Parana-NULL-Curitiba-Barrios-Agua_Verde&tipos=apartamento_residencial,flat_residencial,cobertura_residencial
3
https://www.vivareal.com.br/aluguel/parana/curitiba/bairros/agua-verde/apartamento_residencial/?__vt=srl%3Ab&pagina=3#onde=BR-Parana-NULL-Curitiba-Barrios-Agua_Verde&tipos=apartamento_residencial,flat_residencial,cobertura_residencial
4
https://www.vivareal.com.br/aluguel/parana/curitiba/bairros/agua-verde/apartamento_residencial/?__vt=srl%3Ab&pagina=4#onde=BR-Parana-NULL-Curitiba-Barrios-Agua_Verde&tipos=apartamento_residencial,flat_residencial,cobertura_residencial
5
https://www.vivareal.com.br/aluguel/parana/curitiba/bairros/agua-verde/apartamento_residencial/?__vt=srl%3Ab&pagina=5#onde=BR-Parana-NULL-Curitiba-Barrios-Agua_Verde&tipos=apartamento_residencial,flat_residencial,cobertura_residencial
6
https://www.vivareal.com.br/aluguel/parana/curitib

In [24]:
len(cards_info)

432

In [26]:
len(np.unique(cards_info))

48

In [80]:
url = "https://www.vivareal.com.br/aluguel/parana/curitiba/bairros/agua-verde/apartamento_residencial/?__vt=srl%3Ab#onde=BR-Parana-NULL-Curitiba-Barrios-Agua_Verde&tipos=apartamento_residencial,flat_residencial,cobertura_residencial"
occurrences = []
#session = requests.Session()
response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')
total_results = soup.find('strong', {'class' : 'results-summary__count js-total-records'}).text

cards_info = []
with requests.Session() as session:
    # 36 resultados por pagina, faz o teto para garantir que acessou todas as paginas
    for i in range(1, math.ceil(int(total_results)/36)+1):
        print(i)
        if i != 1:
            url = "https://www.vivareal.com.br/aluguel/parana/curitiba/bairros/agua-verde/apartamento_residencial/?pagina={}&__vt=srl:b".format(i)
        else:
            url = "https://www.vivareal.com.br/aluguel/parana/curitiba/bairros/agua-verde/apartamento_residencial/?__vt=srl%3Ab#onde=BR-Parana-NULL-Curitiba-Barrios-Agua_Verde&tipos=apartamento_residencial,flat_residencial,cobertura_residencial"
        #payload = {'pagina': i}
        response = session.get(url, allow_redirects=True)#, params=payload)
        print(response.url)

        if response.history:
            for resp in response.history:
                print(resp.url)
            print(response.url)
        else:
            print('nao foi redirecionado')
        soup = BeautifulSoup(response.text, 'html.parser')

        cards = soup.find_all('div', 'property-card__main-content')

        for card in cards:
            link_detalhes = card.find('a', 'property-card__title js-cardLink js-card-title')['href']
            id = re.search('id-(\d+)', link_detalhes).group(1)
            if id == '2484870391':
                occurrences.append(i)
            cards_info.append(id)
        file = open('pagina{}.html'.format(i), 'w')
        file.write(response.text)
        file.close()

1
https://www.vivareal.com.br/aluguel/parana/curitiba/bairros/agua-verde/apartamento_residencial/?__vt=srl%3Ab#onde=BR-Parana-NULL-Curitiba-Barrios-Agua_Verde&tipos=apartamento_residencial,flat_residencial,cobertura_residencial
nao foi redirecionado
2
https://www.vivareal.com.br/aluguel/parana/curitiba/bairros/agua-verde/apartamento_residencial/?pagina=2&__vt=srl:b
nao foi redirecionado
3
https://www.vivareal.com.br/aluguel/parana/curitiba/bairros/agua-verde/apartamento_residencial/?pagina=3&__vt=srl:b
nao foi redirecionado
4
https://www.vivareal.com.br/aluguel/parana/curitiba/bairros/agua-verde/apartamento_residencial/?pagina=4&__vt=srl:b
nao foi redirecionado
5
https://www.vivareal.com.br/aluguel/parana/curitiba/bairros/agua-verde/apartamento_residencial/?pagina=5&__vt=srl:b
nao foi redirecionado
6
https://www.vivareal.com.br/aluguel/parana/curitiba/bairros/agua-verde/apartamento_residencial/?pagina=6&__vt=srl:b
nao foi redirecionado
7
https://www.vivareal.com.br/aluguel/parana/curit

In [81]:
occurrences

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [82]:
len(cards_info)

432

In [83]:
np.unique(cards_info)

array(['2449103844', '2463070583', '2464430673', '2472048125',
       '2472157694', '2474123454', '2475596056', '2479753402',
       '2480058484', '2481786996', '2482023424', '2482123616',
       '2483406506', '2484870391', '2485836282', '2486465405',
       '2486651199', '2487249034', '2487525190', '2487622920',
       '2488017106', '2488092854', '2489719861', '2489834202',
       '2490462708', '2490858868', '2490896688', '2490987557',
       '2490999203', '2491073478', '2491910177', '2491969695',
       '2491978023', '2492841297', '2493130017', '2493200089',
       '2493483400', '2493525588', '2493602325', '2493738580',
       '2493880531', '2493901300', '2494065876'], dtype='<U10')

In [33]:
url = "https://www.vivareal.com.br/aluguel/parana/curitiba/bairros/agua-verde/apartamento_residencial/?__vt=srl%3Ab#onde=BR-Parana-NULL-Curitiba-Barrios-Agua_Verde&tipos=apartamento_residencial,flat_residencial,cobertura_residencial"

response = requests.get(url)
response.cookies

<RequestsCookieJar[Cookie(version=0, name='__cfduid', value='d60138e404158cfb8cc11e0d7d9746d0e1599017697', port=None, port_specified=False, domain='.vivareal.com.br', domain_specified=True, domain_initial_dot=True, path='/', path_specified=True, secure=False, expires=1601609697, discard=False, comment=None, comment_url=None, rest={'HttpOnly': None, 'SameSite': 'Lax'}, rfc2109=False), Cookie(version=0, name='__cfruid', value='d5664fcd148bffd78feee61c62bac5d2dd76f079-1599017699', port=None, port_specified=False, domain='.vivareal.com.br', domain_specified=True, domain_initial_dot=True, path='/', path_specified=True, secure=True, expires=None, discard=True, comment=None, comment_url=None, rest={'HttpOnly': None, 'SameSite': 'None'}, rfc2109=False)]>

In [34]:
url = "https://www.vivareal.com.br/aluguel/parana/curitiba/bairros/agua-verde/apartamento_residencial/?__vt=srl%3Ab&pagina=4#onde=BR-Parana-NULL-Curitiba-Barrios-Agua_Verde&tipos=apartamento_residencial,flat_residencial,cobertura_residencial"

response = requests.get(url)
response.cookies

<RequestsCookieJar[Cookie(version=0, name='__cfduid', value='dd796fc4084f967b5d8dde6e1580da28a1599017764', port=None, port_specified=False, domain='.vivareal.com.br', domain_specified=True, domain_initial_dot=True, path='/', path_specified=True, secure=False, expires=1601609764, discard=False, comment=None, comment_url=None, rest={'HttpOnly': None, 'SameSite': 'Lax'}, rfc2109=False), Cookie(version=0, name='__cfruid', value='822acecd49b0c746cb8b51438e37dd7f91deff77-1599017764', port=None, port_specified=False, domain='.vivareal.com.br', domain_specified=True, domain_initial_dot=True, path='/', path_specified=True, secure=True, expires=None, discard=True, comment=None, comment_url=None, rest={'HttpOnly': None, 'SameSite': 'None'}, rfc2109=False)]>